### Import Graph

In [4]:
import networkx as nx

# Load the directed graph from AmazonGraph.txt
G = nx.read_edgelist('AmazonGraph.txt', 
                     create_using=nx.DiGraph(), 
                     nodetype=int,
                     data=False)

print(f"Graph loaded successfully!")
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Graph loaded successfully!
Number of nodes: 50000
Number of edges: 162551


### Centrality Analysis

In [5]:
# Compute PageRank
print("\nComputing PageRank...")
pagerank = nx.pagerank(G, alpha=0.85, max_iter=100)

# Display top 10 nodes by PageRank
print("\nTop 10 nodes by PageRank:")
top_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:10]
for node, score in top_pagerank:
    print(f"  Node {node}: {score:.6f}")


Computing PageRank...

Top 10 nodes by PageRank:
  Node 548091: 0.002062
  Node 502784: 0.001446
  Node 296016: 0.001369
  Node 515301: 0.001338
  Node 265965: 0.001245
  Node 502086: 0.001245
  Node 425735: 0.001225
  Node 335281: 0.001218
  Node 180888: 0.001178
  Node 186356: 0.001136


In [6]:
# Compute Eigenvector Centrality
print("Computing Eigenvector Centrality...")
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-6)

# Display top 10 nodes by Eigenvector Centrality
print("\nTop 10 nodes by Eigenvector Centrality:")
top_eigenvector = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
for node, score in top_eigenvector:
    print(f"  Node {node}: {score:.6f}")

Computing Eigenvector Centrality...

Top 10 nodes by Eigenvector Centrality:
  Node 335281: 0.353086
  Node 75075: 0.288180
  Node 244004: 0.288180
  Node 220967: 0.274488
  Node 359489: 0.274488
  Node 427211: 0.167548
  Node 502086: 0.147663
  Node 148927: 0.128078
  Node 148932: 0.127968
  Node 32862: 0.127446


### Community Detection

In [ ]:
# Identify community structure

### Diffusion or Threshold Modeling

In [ ]:
# Simulate an information or contagion process
# e.g., SI, SIR, or threshold model

### Model Comparison

In [ ]:
# Generate small synthetic networks
# nx.erdos_renyi_graph()
# nx.watts_strogatz_graph()
# nx.barabasi_albert_graph()

### Creative or Extended Analysis

In [ ]:
# Link metadata to nodes and analyze